In [303]:
# Dependencies
import pandas as pd
import json
import numpy as np
import jenkspy as jpy

# TRL - Ranking Algorithm

### This project uses WAR to determine the rankings of specific TRL players. Configuration options for the algorithm can be found in the cell below.

In [304]:
# Configuration Options

# Minimum threshold of games needed for a player in order to receive a rating:
games_threshold = 15

# WAR Calculation multipliers. Must add to 1
offense_multiplier = 0.6
defense_multiplier = 0.3
support_multiplier = 0.1

# Set to False to use classical WAR instead of cWAR
use_cwar = True
# cWAR Coefficient scalar
cwar_scalar = 0.2

# Part 1: Data Wrangling

### In this cell, we can determine which seasons we want to use for WAR computation

In [305]:
# Load all CSV files
Fall2022   = pd.read_csv('csv/Fall2022.csv',   delimiter=';')
Fall2023   = pd.read_csv('csv/Fall2023.csv',   delimiter=';')
Spring2023 = pd.read_csv('csv/Spring2023.csv', delimiter=';')
Spring2024 = pd.read_csv('csv/Spring2024.csv', delimiter=';')
Summer2023 = pd.read_csv('csv/Summer2023.csv', delimiter=';')
Winter2023 = pd.read_csv('csv/Winter2023.csv', delimiter=';')
Winter2024 = pd.read_csv('csv/Winter2024.csv', delimiter=';')

all_seasons = [Fall2022, Winter2023, Spring2023, Summer2023, Fall2023, Winter2024, Spring2024]

# Using all season data:
seasons = all_seasons

# Using only last 3 seasons:
# seasons = [Fall2023, Winter2024, Spring2024]

# Index of first season we want to use (do not modify)
start_season = None
for i, season in enumerate(all_seasons):
    if id(season) == id(seasons[0]):
        start_season = i + 1
        break

### Core merging / data wrangling algorithm. 

#### If additional usernames need to be added for a player, modify the `json/username_mapping.json` file and re-run these cells.

In [306]:
def merge_by_username_per_season(seasons_list):
    """
    Merges and aggregates statistics by player name for each DataFrame in the list separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with statistics aggregated by player name for each season.
    """
    
    # Load JSON data from file into a Python dictionary
    with open('json/username_mapping.json', 'r') as file:
        username_mapping = json.load(file)
    
    def merge_and_aggregate(df):
        # Make all player names lowercase
        df['player name'] = df['player name'].str.lower()
        
        # Replace mapped names with their dictionary value
        df['player name'] = df['player name'].map(username_mapping).fillna(df['player name'])
        
        # Choose the statistics we actually want to use. These are total statistics across a season 
        # (i.e. none of these can be averages of any sort)
        columns_to_aggregate = [
            # Core statistics
            'games', 'wins', 'goals', 'assists', 'saves', 'shots', 
            # Other helpful, but not core statistics
            'shots conceded', 'goals conceded', 'amount stolen', 'amount used while supersonic', 
            # Time statistics
            'time supersonic speed', 'time on ground', 'time low in air', 'time high in air',
            'time in front of ball', 'time behind ball', 'time defensive third', 'time neutral third', 'time offensive third', 
            # Demolition statistics
            'demos inflicted', 'demos taken'
        ]
        
        # Merge all the statistics by adding them all up with respect to the player name
        grouped_by_name = df.groupby('player name')[columns_to_aggregate].sum().reset_index()
        
        return grouped_by_name
    
    # Process each season DataFrame
    merged_seasons = [merge_and_aggregate(season_df) for season_df in seasons_list]
    
    return merged_seasons


def calculate_per_game_per_season(seasons_list, threshold):
    """
    Calculate player statistics per game for each DataFrame in the list separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process.
    games_threshold (int): Minimum number of games required to be included in the calculations.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with per-game statistics for each season.
    """
    
    def calculate_per_game(season):
        # Calculate player statistics per game
        per_game_stats = season.copy()
        
        # Filter anyone with less than minimum required games
        per_game_stats = per_game_stats[per_game_stats['games'] >= threshold]
        
        # Calculate stats per game
        for col in ['wins', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded', 'amount stolen',
                    'amount used while supersonic', 'time supersonic speed', 'time on ground', 'time low in air',
                    'time high in air', 'time in front of ball', 'time behind ball', 'time defensive third',
                    'time neutral third', 'time offensive third', 'demos inflicted', 'demos taken']:
            per_game_stats[col] = per_game_stats[col] / per_game_stats['games']
        
        # Rename wins to winrate now that it is a percentage:
        per_game_stats = per_game_stats.rename(columns={'wins': 'winrate'})
        
        # Drop unneeded columns
        per_game_stats.drop(columns=['games'], inplace=True)
        per_game_stats.reindex()
        
        return per_game_stats
    
    # Process each season DataFrame
    per_game_seasons = [calculate_per_game(season_df) for season_df in seasons_list]
    
    return per_game_seasons

### Data wrangling complete. 

#### All statistics necessary can be found in the `per_game_stats_by_season` variable, which is a list of dataframes sorted by oldest to most recent season.
#### Each dataframe in the list contains statistics-per-game values for that season, eg. Shots per Game for Spring 2024 

In [307]:
merged_seasons_by_username = merge_by_username_per_season(seasons)
per_game_stats_by_season = calculate_per_game_per_season(merged_seasons_by_username, games_threshold)

# Data wrangling complete. The per_game_stats_by_season is our final array which has every useful statistic in it.
# Display the results:
for i, season_df_per_game in enumerate(per_game_stats_by_season, start=start_season):
    print(f"Season {i} merged statistics:")
    print(season_df_per_game)

Season 1 merged statistics:
              player name   winrate     goals   assists     saves     shots  \
0            ant the knee  0.358491  1.188679  0.377358  2.641509  3.679245   
1              asianapple  0.340426  0.574468  0.489362  0.914894  1.638298   
2                brictone  0.550000  1.025000  0.600000  1.225000  2.575000   
3              bylebukong  0.530612  1.163265  0.469388  1.081633  3.387755   
4   caden the fish fisher  0.660714  0.625000  0.607143  0.875000  2.607143   
5                 chyaboi  0.593220  0.966102  0.593220  1.169492  3.084746   
6            commanderboy  0.574074  0.518519  0.500000  1.981481  1.833333   
7            cooldudesood  0.377778  0.133333  0.288889  0.377778  0.488889   
8                    cryo  0.358974  0.076923  0.307692  0.641026  0.384615   
9              dazedrex69  0.404762  0.285714  0.333333  1.238095  1.190476   
10                   deso  0.333333  0.833333  0.714286  1.523810  2.500000   
11                   die

# Part 2: Load team data

### Teammates and their data is sometimes used during WAR computation, so it is loaded here.

In [308]:
# Helpful methods for loading and locating teammate data

def get_statistic(name, statistics_df, statistic_name):
    # Filter the DataFrame for the given player name
    player_stats = statistics_df[statistics_df['player name'] == name]

    # Check if the player exists in the DataFrame
    if not player_stats.empty:
        # Return the desired statistic value
        return player_stats.iloc[0][statistic_name]
    
    # Return 0 if the player does not exist
    return 0

def get_teammate_stats(teams_one_season, statistics_df, statistic_name):
    # 1. create two new columns with the teammate names
    df_with_teammates = statistics_df.copy()
    df_with_teammates['teammate_1'] = ''
    df_with_teammates['teammate_2'] = ''
    
    for index, row in df_with_teammates.iterrows():
        name = row['player name']
        
        for team_list in teams_one_season:
            if name in team_list:
                teammates_list = team_list.copy()
                teammates_list.remove(name)
                
                if len(teammates_list) >= 2:
                    df_with_teammates.at[index, 'teammate_1'] = teammates_list[0]
                    df_with_teammates.at[index, 'teammate_2'] = teammates_list[1]
                elif len(teammates_list) == 1:
                    df_with_teammates.at[index, 'teammate_1'] = teammates_list[0]
        
    # 2. Load the necessary statistics and add to the dataframe
    df_with_teammates[f'teammate_{statistic_name}'] = 0.0
    for index, row in df_with_teammates.iterrows():
        
        df_with_teammates.at[index, f'teammate_{statistic_name}'] = (
                get_statistic(row['teammate_1'], statistics_df, statistic_name) + get_statistic(row['teammate_2'], statistics_df, statistic_name)
        )
        
        # Add any other necessary teammate statistics here...
    
    # Drop unneeded columns
    df_with_teammates.drop(columns=['teammate_1', 'teammate_2'], inplace=True)
    df_with_teammates.reindex()    
    
    return df_with_teammates

# Load the JSON file containing all the teams
with open('json/teams_per_season.json', 'r') as json_file:
    teams_per_season = json.load(json_file)

# Part 3: WAR Computation

### This is the core algorithm that makes the wheels turn. 
#### Note that the offense, defense, and support multipliers can be modified in the configuration cell, near the top of this notebook.

In [309]:
def calculate_war_per_season(seasons_list, offense_mult=offense_multiplier, defense_mult=defense_multiplier, support_mult=support_multiplier):
    """
    Calculate WAR for each player in each season DataFrame separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process. This should be averages per game, sorted by season.
    offense_multiplier (float): Multiplier for offensive statistics.
    defense_multiplier (float): Multiplier for defensive statistics.
    support_multiplier (float): Multiplier for support statistics.
    
    Returns:
    list of pd.DataFrame: List of DataFrames with WAR calculated for each player in each season.
    """
    
    def calculate_war(player_stats_one_season, season_index):
        # Calculate averages for each statistic
        averages_one_season = player_stats_one_season[player_stats_one_season.select_dtypes(include='number').columns].mean()
        
        # Make a new dataframe to store the WAR computations
        rankings_one_season = player_stats_one_season.copy()
        
        # Pull teammate statistics
        teammate_stats_one_season = get_teammate_stats(teams_per_season.get(f"{season_index}"), player_stats_one_season, 'goals')
        
        # Calculate WAR
        rankings_one_season['WAR'] = (
            offense_mult * (
                + (player_stats_one_season['goals'] - averages_one_season['goals'])
                + (player_stats_one_season['assists'] - averages_one_season['assists']) * 0.75
                + (player_stats_one_season['shots'] - averages_one_season['shots']) * 0.33          
            ) +
            defense_mult * (
                + (player_stats_one_season['saves'] - averages_one_season['saves']) * 0.6
                - (player_stats_one_season['shots conceded'] - averages_one_season['shots conceded']) * 0.15
                - (player_stats_one_season['goals conceded'] - averages_one_season['goals conceded']) * 0.33
            ) + 
            support_mult * (
                + (player_stats_one_season['demos inflicted'] - averages_one_season['demos inflicted']) * 0.1
                - (player_stats_one_season['demos taken'] - averages_one_season['demos taken']) * 0.1
                + (player_stats_one_season['amount stolen'] - averages_one_season['amount stolen']) * 0.005
            )
        ) + 1
                        
        # Drop every column except the ones we want to view
        rankings_one_season = rankings_one_season[['player name', 'WAR']]
        
        # Sort by WAR
        rankings_one_season = rankings_one_season.sort_values(by='WAR', ascending=False)
        
        return rankings_one_season
    
    # Process each season DataFrame
    war_seasons = [calculate_war(season_df, i) for i, season_df in enumerate(seasons_list, start=start_season)]
    
    return war_seasons

In [310]:
# Run the calculate_war_per_season function
war_by_season = calculate_war_per_season(per_game_stats_by_season)

# Display the results
for i, season_df in enumerate(war_by_season, start=start_season):
    print(f"Season {i} WAR statistics:")
    print(season_df)

Season 1 WAR statistics:
              player name       WAR
28                   mini  2.240368
26               lavablue  1.881677
41                shaunch  1.879586
45                  steve  1.844715
36           rubber ducky  1.802172
16                  goofy  1.783325
35                    rex  1.693000
0            ant the knee  1.662560
17           greensleeves  1.641937
3              bylebukong  1.610726
47             terminator  1.514647
38            scootleboot  1.499620
46                  swej.  1.470927
5                 chyaboi  1.420854
43                   solo  1.413793
14                fernado  1.392277
18                hotshot  1.379951
2                brictone  1.275252
48             vt stormzy  1.234085
10                   deso  1.136527
44                 sooper  1.132179
13                 dreamz  1.069022
19                  idoit  1.032402
11                   dies  1.022575
4   caden the fish fisher  0.999608
30            neosunlight  0.980614
6  

# Part 3b: Experimental Clustering

### This is a test cell which uses the Fisher-Jenks algorithm to assign predicted tiers to players.

In [320]:
for i, season_df in enumerate(war_by_season, start=start_season):
    breaks = jpy.jenks_breaks(season_df['WAR'], n_classes=5)
    season_df['predicted_tier'] = pd.cut(season_df['WAR'], bins=breaks, labels=['tier5', 'tier4', 'tier3', 'tier2', 'tier1'])
    
    print(f"Season {i} clusters:")
    print(season_df)

Season 1 clusters:
              player name       WAR predicted_tier
28                   mini  2.240368          tier1
26               lavablue  1.881677          tier1
41                shaunch  1.879586          tier1
45                  steve  1.844715          tier1
36           rubber ducky  1.802172          tier1
16                  goofy  1.783325          tier1
35                    rex  1.693000          tier1
0            ant the knee  1.662560          tier1
17           greensleeves  1.641937          tier1
3              bylebukong  1.610726          tier2
47             terminator  1.514647          tier2
38            scootleboot  1.499620          tier2
46                  swej.  1.470927          tier2
5                 chyaboi  1.420854          tier2
43                   solo  1.413793          tier2
14                fernado  1.392277          tier2
18                hotshot  1.379951          tier2
2                brictone  1.275252          tier2
48          

# Part 4: Corrected WAR (cWAR)

### This sub-algorithm factors in performance of teammates to award bonuses for "carry" potential.

In [312]:
def calculate_cwar_per_season(seasons_list):
    """
    Calculate cWAR for each player in each season DataFrame separately.
    
    Parameters:
    seasons (list of pd.DataFrame): List of DataFrames to process. This should be WAR per player, sorted by season.
    scaling_factor (float): Multiplier for cWAR computation
    
    Returns:
    list of pd.DataFrame: List of DataFrames with cWAR calculated for each player in each season.
    """
    def calculate_cwar(player_war_one_season, season_index, scaling_factor=cwar_scalar):
        
        # Make a new dataframe which has the sum of WAR of both teammates:
        cwar_one_season = get_teammate_stats(teams_per_season.get(f"{season_index}"), player_war_one_season, 'WAR')
        
        # Drop rows where teammate's WAR sum is zero. This usually happens when a player was a sub i.e. the player
        # had no teammates.
        cwar_one_season = cwar_one_season[cwar_one_season['teammate_WAR'] != 0]
        
        # Compute I_p~t for each player, and put it in a column called 'performance'
        cwar_one_season['performance'] = cwar_one_season['WAR'] / cwar_one_season['teammate_WAR']
        
        # Calculate the average across the 'performance' column
        averages_one_season = cwar_one_season[cwar_one_season.select_dtypes(include='number').columns].mean()
        
        # Add statistic to dataframe, for viewing later - line can be commented out to de-clutter final dataframe
        cwar_one_season['avg_performance'] = averages_one_season['performance']
        
        # Finally, calculate the cWAR of each player
        cwar_one_season['cWAR'] = (
                cwar_one_season['WAR'] 
                + scaling_factor * (
                        (cwar_one_season['performance'] - averages_one_season['performance']) / averages_one_season['performance']
                )
        )
        
        return cwar_one_season
        
    
    # Process each season DataFrame
    cwar_seasons = [calculate_cwar(season_df, i) for i, season_df in enumerate(seasons_list, start=start_season)]
    
    return cwar_seasons

In [313]:
# Run the calculate_cwar_per_season function
cwar_by_season = calculate_cwar_per_season(war_by_season)

# Display the results
for i, season_df in enumerate(cwar_by_season, start=start_season):
    print(f"Season {i} cWAR statistics:")
    print(season_df)
    season_df.to_csv(f'results/cwar_season_{i}', index=False)

Season 1 cWAR statistics:
              player name       WAR predicted_tier  teammate_WAR  performance  \
28                   mini  2.240368          tier3      1.063982     2.105645   
26               lavablue  1.881677          tier3      0.994770     1.891569   
41                shaunch  1.879586          tier3      1.207767     1.556248   
45                  steve  1.844715          tier3      2.032010     0.907828   
16                  goofy  1.783325          tier3      0.881330     2.023447   
35                    rex  1.693000          tier3      1.852733     0.913785   
0            ant the knee  1.662560          tier3      0.473196     3.513474   
17           greensleeves  1.641937          tier3      0.868384     1.890795   
3              bylebukong  1.610726          tier3      1.753677     0.918485   
47             terminator  1.514647          tier3      1.419471     1.067050   
38            scootleboot  1.499620          tier3      1.520230     0.986442   
46

# Part 5: WAR Weighting

### More recent seasons will receive a preferential weighting compared to older seasons.
### TODO: This cell currently runs an average across all seasons. Must be modified.

In [314]:
# Add weights for more recent seasons

# Construct a dictionary to store weighted WAR values for each player
# Dictionary format is - 'player name' : [list, of, WARs]
player_war_dict = {}

all_wars = pd.concat(cwar_by_season)
for _, row in all_wars.iterrows():
    if not row['player name'] in player_war_dict:
        player_war_dict[row['player name']] = []
    if use_cwar:
        player_war_dict[row['player name']].append(row['cWAR'])
    else:
        player_war_dict[row['player name']].append(row['WAR'])

for player_name in player_war_dict: print(f"{player_name} : {player_war_dict[player_name]}")

# Now, weight the WARs such that most recent ratings will be favored more
weighted_war_dict = {}
for player_name in player_war_dict:
    weighted_war_dict[player_name] = np.mean(player_war_dict[player_name])

mini : [2.6158597034008624, 2.8854900141575244, 2.725584673550497, 2.5384118828702498, 2.371324349299428]
lavablue : [2.1986599592092806, 1.6276771180341227]
shaunch : [2.1049222348477405]
steve : [1.8928320946206554]
goofy : [2.136350917819186]
rex : [1.742746117857115]
ant the knee : [2.4228238927208525]
greensleeves : [1.9587079389843467, 2.442228532486373, 2.000975090454813, 1.4758356723208936]
bylebukong : [1.6617563608793284, 1.4407919352650649, 1.0751385042993788, 1.4097706566648134]
terminator : [1.6062815813445355, 1.5209621335136438, 1.4386647551418885, 1.8614963238298816, 1.5618943736918778, 1.9401563205086594, 1.472950933056056]
scootleboot : [1.5692233123953794, 1.6550961914315425]
swej. : [1.4942076971435378]
chyaboi : [1.45742677999784, 1.1445098926395516, 1.5937159876169413]
solo : [2.272805773940748, 2.8253566403456043]
fernado : [1.3689800230444995, 1.5779306162825426, 1.0836437198529993]
hotshot : [1.3760053472603324, 1.279621193003837, 2.1408632003537837, 1.50498024

In [315]:
# Convert dictionary to DataFrame
final_weighted_war = pd.DataFrame.from_dict(weighted_war_dict, orient='index', columns=['WAR'])

# Reset index to make player names a column
final_weighted_war.reset_index(inplace=True)

# Rename the columns
final_weighted_war.rename(columns={'index': 'player name'}, inplace=True)

# Sort by WAR and print final results to a csv
final_weighted_war = final_weighted_war.sort_values(by='WAR', ascending=False)
final_weighted_war.to_csv('results/final_war', index=False)

# Part 6: Team Creation

#### This is using a greedy algorithm to create teams. Essentially, each team tries to make the highest total WAR team that they possibly can. Each team picks the highest rated player out of the remaining players. Then, whichever team has the lowest total WAR gets to pick next.

#### Note that this is not a perfect, be-all-end-all solution as it does not exhaustively test all combinations of teams. However, it does get pretty close.

In [322]:
# Signups for Spring 2024. Mostly used as a test.
# playerlist = ['kade', 'mini', 'peak', 'leon', 'snipey', 'greensleeves', 'terminator', 'bylebukong', 'hotshot', 'rubber ducky', 'deso',
#                      'leagueson', 'vpr.vnm', 'tipsy', 'ral days', 'argon', 'fernado', 'pops', 'capi', 'senor brightside', 'arby', 'toucan', 
#                      'tophatbear', 'wika', 'testie', 'waycey', 'king', 'awe', 'phrez', 'front flip freddy', 'lukethighwalkr4', 'dies', 'g_llama',
#                      'renshirokamazaki', 'elatedthug', 'beeholder', 'mistermirz', 'uday', 'gangster.goose']

# Signups for Summer 2024 (so far)
playerlist = ['leagueson', 'deso', 'tortle', 'testie', 'argon', 'snipey', 'lukethighwalkr4', 'fernado', 'elatedthug', 'penquan', 'vpr.vnm', 'idoit', 'king',
              'tipsy', 'g_llama', 'sales', 'toucan', 'front flip freddy', 'rubber ducky', 'luma', 'domo', 'kail', 'renshirokamazaki', 'roo', 'jmc', 'greensleeves', 
              'hotshot', 'pastamafia']

playerlist = final_weighted_war[final_weighted_war['player name'].isin(playerlist)]

# Sort by WAR
playerlist = playerlist.sort_values(by='WAR', ascending=False)

breaks = jpy.jenks_breaks(playerlist['WAR'], n_classes=4)
playerlist['predicted_tier'] = pd.cut(playerlist['WAR'], bins=breaks, labels=['tier4', 'tier3', 'tier2', 'tier1'])

print(f"clusters:")
print(playerlist)

players = playerlist.to_dict('records')

# Initialize teams
teams = [[] for _ in range(len(playerlist) // 3)]
team_wars = [0] * (len(playerlist) // 3)

# List to store full teams
full_teams = []

# Assign players to teams greedily
for player in players:
    if len(teams) == 0:
        break  # If all teams are already full, break the loop
    
    # Find the team with the lowest WAR
    best_team_index = np.argmin(team_wars)
    teams[best_team_index].append(player)
    team_wars[best_team_index] += player['WAR']
    
    # Check if the team is full (3 players)
    if len(teams[best_team_index]) == 3:
        full_teams.append(teams[best_team_index])
        teams.pop(best_team_index)
        team_wars.pop(best_team_index)

# Display the full teams
print("Full Teams:")
for i, team in enumerate(full_teams):
    print(f"Team {i+1}:")
    for player in team:
        print(f"\t{player['player name']} (WAR: {player['WAR']})")
    print(f"\t\tTeam WAR: {sum(player['WAR'] for player in team)}")

# If there are any incomplete teams left, display them as well
if teams:
    print("\nIncomplete Teams:")
    for i, team in enumerate(teams):
        print(f"Incomplete Team {i+1}:")
        for player in team:
            print(f"\t{player['player name']} (WAR: {player['WAR']})")
        print(f"\t\tTeam WAR: {sum(player['WAR'] for player in team)}")

clusters:
           player name       WAR predicted_tier
7         greensleeves  1.969437          tier1
100             snipey  1.661077          tier1
50        rubber ducky  1.647555          tier1
15             hotshot  1.559638          tier1
14             fernado  1.343518          tier2
18                deso  1.322103          tier2
80               tipsy  1.275329          tier2
77               argon  1.217050          tier2
67           leagueson  1.187865          tier2
68             vpr.vnm  1.186052          tier2
53              toucan  0.978491          tier2
21               idoit  0.728661          tier3
57                luma  0.714313          tier3
29             penquan  0.666188          tier3
104             testie  0.580132          tier3
41                king  0.444098          tier4
82   front flip freddy  0.430278          tier4
86     lukethighwalkr4  0.345426          tier4
73             g_llama  0.303295          tier4
72                 roo  0.2536